In [ ]:
!pip install lda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 351 kB 5.1 MB/s 
     |████████████████████████████████| 99 kB 9.0 MB/s 


In [ ]:
import nltk
import re
import collections
import itertools
import lda
import requests
import csv
import time
import math
import operator
import numpy as np
import pandas as pd
from collections import defaultdict
from pandas import read_table
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB # 다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score    # 정확도 계산

In [ ]:
df = pd.read_csv('disney.csv')
df

,Unnamed: 0,rating,review,date,userName,title
0,0,5,컨텐츠야 디즈니니까 당연히 넘쳐나는데 검색 기능이 상대적으로 부실합니다. 예를 들면...,2021-11-11 20:33:32,악틱,좋습니당. 몇 가지 건의 사항
1,1,5,디즈니 만화와 영화를 사랑하는 사람으로써 몇 가지 개선해주셨으면 하는 바가 있어 남...,2021-11-15 12:15:12,이제가을,디즈니를 아끼는 마음으로 남겨요~
2,2,5,1년권 끊고 한 달만에 후회했는데 문나이트로 뽕 뽑음. 문나이트만 한 다섯번째 돌...,2022-06-06 05:58:35,양털끄트머리,하나만 걸려도 충분하다
3,3,3,전반 적으로 만족 합니다만 넷플이나 기타 스트리밍 앱에서 되는 것들이 안되는게 있어...,2021-11-14 06:14:34,filco518,기대하던 디플 앱 드디어 사용 해봤네요
4,4,5,성인인증 시 프로필 PIN 넘버 입력 키보드 자판이 안뜨고 구글 크롬캐스트4 사용 ...,2021-11-11 23:59:48,무무니,컨텐츠 많아서 좋긴한데
...,...,...,...,...,...,...
4535,3457,5,짱,2022-03-12 20:00:39,김지혜,NaN
4536,3458,5,👍,2022-01-14 16:35:11,정윤재,NaN
4537,3459,1,Good 😉😉😉😉😉😉😉😉😉😉😉,2021-03-23 13:07:45,Jacob Won,NaN
4538,3460,1,1000000000000000000000000000000000000000000000...,2022-01-24 21:45:05,황은비,NaN


In [ ]:
df[df['review'].str.contains('뭉탱이') == True]['review']

1050    뭉탱이로있다가유리게슝아니그냥뭉탱이로있다가유리게슝아니그냥뭉탱이로있다가유리게슝아니그냥뭉...
Name: review, dtype: object

In [ ]:
df[df['review'].str.contains('레이디') == True]['review']

986    왜 레이디버그 없어요?  왜 레이디버그  왜 레이디버그 없어요? 왜 레이디버그 없어...
Name: review, dtype: object

In [ ]:
data_list = df['review']
data_list

0       컨텐츠야 디즈니니까 당연히 넘쳐나는데 검색 기능이 상대적으로 부실합니다. 예를 들면...
1       디즈니 만화와 영화를 사랑하는 사람으로써 몇 가지 개선해주셨으면 하는 바가 있어 남...
2       1년권 끊고 한 달만에 후회했는데 문나이트로 뽕 뽑음.  문나이트만 한 다섯번째 돌...
3       전반 적으로 만족 합니다만 넷플이나 기타 스트리밍 앱에서 되는 것들이 안되는게 있어...
4       성인인증 시 프로필 PIN 넘버 입력 키보드 자판이 안뜨고 구글 크롬캐스트4 사용 ...
                              ...                        
4535                                                    짱
4536                                                    👍
4537                                     Good 😉😉😉😉😉😉😉😉😉😉😉
4538    1000000000000000000000000000000000000000000000...
4539                                            👍👍👍👍👍👍👍👍👍
Name: review, Length: 4540, dtype: object

In [ ]:
pos_list = df[df['rating'] >= 4]
neg_list = df[df['rating'] <= 2]

In [ ]:
pos_corpus = np.array(pos_list['review'])
neg_corpus = np.array(neg_list['review'])

In [ ]:
total_data = np.concatenate((pos_corpus,neg_corpus))

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [824 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
from konlpy.tag import Okt
import nltk

In [ ]:
okt=Okt()

In [ ]:
pos_reviews = ''
   
for i in pos_list['review'] : 
  pos_reviews = pos_reviews + i + '\n'

In [ ]:
tokens = okt.morphs(pos_reviews)

In [ ]:
result = nltk.Text(tokens)

In [ ]:
review_pos = okt.pos(pos_reviews)

In [ ]:
pos_df = pd.DataFrame(review_pos)
pos_df.columns = ['token', 'pos']
pos_df.head()

,token,pos
0,컨텐츠,Noun
1,야,Josa
2,디즈니,Noun
3,니까,Josa
4,당연히,Adjective


In [ ]:
noun = pos_df[pos_df['pos']== 'Noun']['token']
josa = pos_df[pos_df['pos']== 'Josa']['token']
verbprefix = pos_df[pos_df['pos']== 'Verbprefix']['token']
verb = pos_df[pos_df['pos']== 'Verb']['token']
punctuation = pos_df[pos_df['pos']== 'Punctuation']['token']
adjective = pos_df[pos_df['pos']== 'Adjective']['token']
suffix = pos_df[pos_df['pos']== 'Suffix']['token']
number = pos_df[pos_df['pos']== 'Number']['token']
adverb = pos_df[pos_df['pos']== 'Adverb']['token']
conjunction = pos_df[pos_df['pos']== 'Conjunction']['token']
alpha = pos_df[pos_df['pos']== 'Alpha']['token']
foreign = pos_df[pos_df['pos']== 'Foreign']['token']
determiner = pos_df[pos_df['pos']== 'Determiner']['token']
modifier = pos_df[pos_df['pos']== 'Modifier']['token']
koreanparticle = pos_df[pos_df['pos']== 'KoreanParticle']['token']
eomi = pos_df[pos_df['pos']== 'Eomi']['token']
exclamation = pos_df[pos_df['pos']== 'Exclamation']['token']
url = pos_df[pos_df['pos']== 'URL']['token']
hashtag = pos_df[pos_df['pos']== 'Hashtag']['token']
email = pos_df[pos_df['pos']== 'Email']['token']
preeomi = pos_df[pos_df['pos']== 'PreEomi']['token']
screenname = pos_df[pos_df['pos']== 'ScreenName']['token']

In [ ]:
noun_list = noun.to_list()

In [ ]:
# 불용어 처리

In [ ]:
total_data.shape

(3959,)

In [ ]:
c_vetorizer = CountVectorizer(analyzer='word')
count = c_vetorizer.fit_transform(noun_list)

In [ ]:
model = lda.LDA(n_topics = 4, n_iter = 1000)
model.fit(count)

In [ ]:
topic_vocab = c_vetorizer.get_feature_names()
topic_word = model.topic_word_
n_top_word = 300
dist = []
for i, topic_dist in enumerate(topic_word):
    dist.append(topic_dist)
    topic_words = np.array(topic_vocab)[np.argsort(topic_dist)][:-n_top_word:-1]
    print('Topic', i+1, topic_words)

Topic 1 ['기능' '버그' '마블' '설정' '화면' '조절' '개선' '한국' '최고' '다음' '시즌' '넷플' '사람' '생각'
 '업데이트' '다시' '크게' '속도' '지원' '서비스' '모바일' '때문' '오류' '가입' '선택' '확인' '그래픽'
 '배경' '별로' '부탁' '계정' '로키' '그거' '심슨' '거의' '미러링' '필요' '비교' '설강화' '상태' '휴대폰'
 '아예' '프로그램' '로그인' '디플' '단점' '표시' '문의' '무조건' '플레이' '굿굿' '로딩' '그게' '요즘'
 '컴퓨터' '다비' '원래' '자꾸' '윈터' '고민' '왓이프' '공개' '이상' '자주' '중국어' '전환' '리안' '예고편'
 '위로' '검은색' '거나' '캐스트' '패드' '연동' '내용물' '스마트' '결재' '치면' '사실' '등록' '건가' '마자'
 '사진' '하자' '오프닝' '화로' '가족' '중이' '회차' '디즈니채널' '솔져' '일본어' '악플' '실수' '이해'
 '아이맥스' '피드백' '아이콘' '안드로이드' '플레이어' '오프라인' '평점' '야호' '어제' '얼마' '흐흐' '웨이'
 '알럽' '나머진' '새벽' '과정' '내년' '퀄리티' '단편' '덕분' '친구' '히어로' '채팅' '릭스' '진행' '감기'
 '광팬' '작업' '뭔가' '무슨' '겨울왕국' '방영' '적용' '환상' '한지' '정작' '공간' '제각각' '캡틴아메리카'
 '쿠폰' '코로나' '울트라' '정상' '정이' '일기' '제작자' '지도' '갓즈니' '의견' '정지' '전화' '시일'
 '강아지' '미키' '며칠' '머리' '만해' '만원' '마침' '구독료' '리즈' '로서' '런가' '또한' '드가자' '동안'
 '대충' '그동안' '극장' '나라' '난후' '무크' '반성' '연속' '번역가' '여러가지' '에어플레이' '앤트맨'
 '알고리즘' '안함' '아이디' '개월' '세대' '걱정' '상담' '사이' '타이밍' '부

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
model.topic_word_

array([[4.96659962e-06, 4.96659962e-06, 4.96659962e-06, ...,
        5.01626561e-04, 5.01626561e-04, 9.98286523e-04],
       [4.27937021e-03, 9.54665273e-04, 3.32945451e-03, ...,
        4.74957847e-06, 4.74957847e-06, 4.74957847e-06],
       [4.74282056e-06, 4.74282056e-06, 4.74282056e-06, ...,
        4.74282056e-06, 4.74282056e-06, 4.74282056e-06],
       [4.88651079e-06, 4.88651079e-06, 4.88651079e-06, ...,
        4.88651079e-06, 4.88651079e-06, 4.88651079e-06]])

In [ ]:
stopword=['디즈니','왜','좀','수','볼','것','그냥','슝','더','땡','보고','점','거','진짜','요','다른','안','이','제','저','그','등','내','시','개','전','폰','뭉탱이','바로',
'다시','달','계속','아주','경우','뭐','정도','일단','앞','한번','때문','못','게','도','언제','편','레이디','유리',
'나','중','함','다만','정','이건','꼭','디','분','외','걸','흠','등등','후','로','완전','이제','그거','매우','건','여기','애','또','지금','은','아니','있다가','입니다','있는','같아요','있어서','부탁드립니다','좋겠습니다','있는데','같습니다','같은','있습니다','있게','있었으면','있어','있고',
'어떻게','같네요']

In [ ]:
from collections import defaultdict
from pandas import read_table
import numpy as np
import math

class NaiveBayesClassifier:

    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []

    def load_corpus(self, path):
        corpus = read_table(path, sep=',', encoding='utf-8')
        corpus = np.array(corpus)
        return corpus

    def count_words(self, training_set):
        # 학습데이터는 영화리뷰 본문(doc), 평점(point)으로 구성
        counts = defaultdict(lambda : [0, 0])
        for doc, point in training_set:
            # 영화리뷰가 text일 때만 카운트
            if self.isNumber(doc) is False:
                

                # 리뷰를 띄어쓰기 단위로 토크나이징
                words = doc.split()

                data_pos = okt.pos(words)
                words_nnp = [word for word,pos in data_pos if pos in ['Noun','Adjective']]
                words = [w for w in words_nnp if not w in stopword]



                for word in words:
                    counts[word][0 if point > 3.5 else 1] += 1
        return counts

    def isNumber(self, s):
        try:
            float(s)
            return True
        except ValueError:
            return False

    def word_probabilities(self, counts, total_class0, total_class1, k):
        # 단어의 빈도수를 [단어, p(w|긍정), p(w|부정)] 형태로 반환
        return [(w,
                 (class0 + k) / (total_class0 + 2*k),
                 (class1 + k) / (total_class1 + 2*k))
                for w, (class0, class1) in counts.items()]

    def class0_probability(self, word_probs, doc):
        # 별도 토크나이즈 안하고 띄어쓰기로만
        docwords = doc.split()

        # 초기값은 모두 0으로 처리
        log_prob_if_class0 = log_prob_if_class1 = 0.0

        # 모든 단어에 대해 반복
        for word, prob_if_class0, prob_if_class1 in word_probs:
            # 만약 리뷰에 word가 나타나면
            # 해당 단어가 나올 log 확률을 더해 줌
            if word in docwords:
                log_prob_if_class0 += math.log(prob_if_class0)
                log_prob_if_class1 += math.log(prob_if_class1)

            # 만약 리뷰에 word가 나타나지 않는다면
            # 해당 단어가 나오지 않을 log 확률을 더해 줌
            # 나오지 않을 확률은 log(1-나올 확률)로 계산
            else:
                log_prob_if_class0 += math.log(1.0 - prob_if_class0)
                log_prob_if_class1 += math.log(1.0 - prob_if_class1)

        prob_if_class0 = math.exp(log_prob_if_class0)
        prob_if_class1 = math.exp(log_prob_if_class1)
        return prob_if_class0 / (prob_if_class0 + prob_if_class1)

    def train(self, trainfile_path):
        training_set = self.load_corpus(trainfile_path)

        # 범주0(긍정)과 범주1(부정) 문서 수를 세어 줌
        num_class0 = len([1 for _, point in training_set if point > 3.5])
        num_class1 = len(training_set) - num_class0

        # train
        word_counts = self.count_words(training_set)
        self.word_probs = self.word_probabilities(word_counts,
                                                  num_class0,
                                                  num_class1,
                                                  self.k)

    def classify(self, doc):
        return self.class0_probability(self.word_probs, doc)

In [ ]:
disney = pd.read_csv('disney.csv')

In [ ]:
disney = disney[['review', 'rating']]

In [ ]:
disney.to_csv('filtered.csv',index=None,header=None)

In [ ]:
model = NaiveBayesClassifier()
model.train(trainfile_path='filtered.csv')

AssertionError: ignored

In [ ]:
model.classify('넷플릭스가 더 좋은듯')

0.8291417013361893

In [ ]:
# 긍정 단어 가중치로 딕셔너리 만들기
dict_p = {}
for i in model.word_probs:
    dict_p[i[0]] = i[1]
# 부정 단어 가중치로 딕셔너리 만들기    
dict_n = {}
for i in model.word_probs:
    dict_n[i[0]] = i[2]
# 각 딕셔너리 값을 내림차순 정렬
sort_dict_p = sorted(dict_p.items(), key=operator.itemgetter(1), reverse=True)
sort_dict_n = sorted(dict_n.items(), key=operator.itemgetter(1), reverse=True)

In [ ]:
# 각 리뷰별 상위 단어 추출
for i in sort_dict_p[0:30]:
    print(i[0])

너무
왜
디즈니
레이디버그
자막
수
다
좀
좋아요
없어요?
볼
자막이
마블
디즈니!디즈니!디즈니!디즈니!
더
잘
좋은데
많이
보고
정말
있는
진짜
다른
굿
그리고
것
기능
제발
있어서
배속


In [ ]:
for i in sort_dict_n[0:30]:
    print(i[0])

너무
자막
왜
좀
안
자막이
앱
디즈니
다
진짜
계속
다른
수
다시
컨텐츠
그리고
볼
잘
어플
없고
더
그냥
보고
영상
앱이
많이
안되고
했는데
아니
제대로
